In [1]:
import os

from flask import Flask

from pywebio.input import *
from pywebio.output import *
from pywebio import start_server
import time
import re
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

CHROMEDRIVER_PATH = '/usr/local/bin/chromedriver'
s = Service(CHROMEDRIVER_PATH)
WINDOW_SIZE = "1920,1080"

# Options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome(#service=s,
    options=chrome_options)


In [2]:
app = Flask(__name__)

env_config = os.getenv("APP_SETTINGS", "config.DevelopmentConfig")

app.config.from_object(env_config)
@app.route("/")

gene_aliases = []
marker_aliases = []


def app():

    put_markdown('## GeneCard alias generator')
    #put_text("Please enter the gene name:")
    gene = input("GeneCards name", type=TEXT, required = True)
    
    put_processbar('bar', auto_close=True)
    for i in range(1, 11):
        set_processbar('bar', i / 10)
        time.sleep(0.1)
        
    get_webpage(gene)
    
    process_aliases(gene_aliases)

    marker_aliases2 = list(set(marker_aliases))
    res = [i for n, i in enumerate(marker_aliases) if i not in marker_aliases[:n]]
    put_success("Done")
    put_text(str(gene_aliases[0]) + " aliases:").style('color: black; font-size: 20px')
    put_text(*gene_aliases, sep =', ')
    put_text("Markers:").style('color: black; font-size: 20px')
    put_text(*res, sep =', ')
    clear(gene_aliases)
    clear(marker_aliases)
    #clear(gene)


SyntaxError: invalid syntax (2417086203.py, line 9)

In [ ]:
def get_webpage(gene):
    try:
        put_processbar('bar', auto_close=True)
        for i in range(1, 11):
            set_processbar('bar', i / 10)
            time.sleep(0.1)
    
        o = driver.get("https://www.genecards.org/cgi-bin/carddisp.pl?gene="+str(gene))
        content = driver.page_source
        soup = BeautifulSoup(content)
        r = driver.current_url 

        
        for a in soup.find('span', attrs={'class':'aliasMainName'}):
            gene_aliases.append(a.text)
 
        
        for a in soup.find_all('span', itemprop = 'alternateName'):
            gene_aliases.append(a.text)
        
        for i in range(len(gene_aliases)):
            gene_aliases[i] = gene_aliases[i].upper()
            
        if str(gene.upper()) not in str(r.upper()):
            put_text("I did not find what you were looking for, but here are results for " + str(gene_aliases[0])).style('color: purple; font-size: 30px')
            
    except Exception as e: 
        put_text("Wrong gene name").style('color: red; font-size: 30px')
        raise
   
    

In [ ]:
def clear(lista):
    lista.clear()

In [ ]:
def process_aliases(gene_aliases):
    put_processbar('bar', auto_close=True)
    for i in range(1, 11):
        set_processbar('bar', i / 10)
        time.sleep(0.1)
        
    for j in gene_aliases:
        if bool(re.search(r'\d', j)) == True:
            j1 = re.sub("[A-Z]+", lambda ele: "" + ele[0] + "_",j)
        else:
            j1=j
        j1=j1.replace("-", "?")
        j1=j1.replace("_?", "?")
        marker_aliases.append(j1)

In [ ]:
# main function
if __name__ == '__main__':
	start_server(app, port=32420, debug=True)